In [ ]:
import numpy as np
NDIGIT = 10 # rounding / precision limit we care about

In [ ]:
def inner(u, v):
    return u.T @ METRIC @ v

def orthogonalize(vs):
    dpluss, _ = METRIC.shape
    n, dps = vs.shape
    assert n <= dpluss
    assert dps == dpluss
    us = np.zeros_like(vs)
    for i in range(n):
        ui = 1. * vs[i]
        for j in range(i):
            ui -= (inner(vs[i], us[j]) / inner(us[j], us[j])) * us[j]
        us[i] = ui
    return us

In [ ]:
s, d = 1, 2 # Lorentz space
foo = np.ones(d + s)
foo[s:] = -1.
METRIC = np.diag(foo)
print(METRIC)

In [ ]:
# make adversarial examples by hand
a1 = np.array([4.8, 3.0, 4.0])
a2 = np.array([0.4, 2.0, -2.0])
for a in [a1, a2]:
    print(np.round(inner(a, a), NDIGIT))

In [ ]:
# this is bad
us = orthogonalize(np.array([a1, a2]))
for u in us:
    print(np.round(inner(u, u), NDIGIT))

In [ ]:
# perhaps suprisingly, this is also bad
us = orthogonalize(np.array([a2, a1]))
for u in us:
    print(np.round(inner(u, u), NDIGIT))

In [ ]:
# even linear combinations bork!
rng = np.random.default_rng(17)
TINY = 10 ** -NDIGIT
ntrial = 2 ** 3
nsuccess = 0
for t in range(ntrial):
    ras = rng.normal(size=(2, 2)) @ np.array([a1, a2])
    us = orthogonalize(ras)
    nsuccess += 1
    if np.abs(np.prod([inner(u, u) for u in us])) < TINY:
        print(t, "failed on ", ras, us, [np.round(inner(u, u), NDIGIT) for u in us])
        nsuccess -= 1
print("Out of", ntrial, "trials:", nsuccess, "successes")

In [ ]:
# and yet: You never hit nulls randomly
rng = np.random.default_rng(17)
ntrial = 2 ** 16
nsuccess = 0
for t in range(ntrial):
    ras = rng.normal(size=(2, 3))
    us = orthogonalize(ras)
    nsuccess += 1
    if np.abs(np.prod([inner(u, u) for u in us])) < TINY:
        print(t, "failed on ", ras, us, [inner(u, u) for u in us])
        nsuccess -= 1
print("Out of", ntrial, "trials:", nsuccess, "successes")